In [ ]:
pip install hazm

     |████████████████████████████████| 317kB 8.1MB/s 
     |████████████████████████████████| 235kB 36.0MB/s 
     |████████████████████████████████| 1.4MB 31.0MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153926 sha256=c5b6d5891056b934cae7dcda9d8755a5195d5133d5f3b5215a7154ce152581c6
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394469 sha256=59d424a073a1c2816328c1eee228c4f24d9831f0c849382279331fc0a65702b7
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


# **load and preprocess text data**

 **load train data**

In [ ]:
#import the librarise

import numpy as np
import pandas as pd
import glob
import os
from __future__ import unicode_literals
from hazm import *

# tokenization and normalization train data

os.chdir(r'/content/drive/MyDrive/train')
myFiles = glob.glob('*.txt')

def readfile(yourfile):

    a=[]
    for file in yourfile:
        fo = open("/content/drive/MyDrive/train/"+file, "r")
        a.append(fo.read())
    return a
text=readfile(myFiles)
normalizer = Normalizer()
train=[word_tokenize(normalizer.normalize(i)) for i in text]


In [ ]:
#normalization and tokenization test data

os.chdir(r'/content/drive/MyDrive/test')
myFiles = glob.glob('*.txt')

def readfile(yourfile):

    a=[]
    for file in yourfile:
        fo = open("/content/drive/MyDrive/test/"+file, "r")
        a.append(fo.read())
    return a
text=readfile(myFiles)
normalizer = Normalizer()
test=[word_tokenize(normalizer.normalize(i)) for i in text]

# **n_garam language model**

**unigaram**

# **for add K smoothing**

In [ ]:
def count_words(tokenized_doc):
    """
    Count the number of word appearence in the tokenized sentences

    Args:
        tokenized_doc: List of lists of strings

    Returns:
        dict that maps word (str) to the frequency (int)

    in this step pass the document to the function and build dictionary that hold the count of word
    """

    word_counts = {}


    # Loop through each sentence
    for doc in tokenized_doc:

        # Go through each token in the sentence
        for token in doc:

            # If the token is not in the dictionary yet, set the count to 1
            if token not in word_counts.keys():
                word_counts[token] = 1

            # If the token is already in the dictionary, increment the count by 1
            else:
                word_counts[token] += 1

    return word_counts

In [ ]:
def get_words_with_nplus_frequency(tokenized_doc, count_threshold):
    """
    Find the words that appear N times or more

    Args:
        tokenized_doc: List of lists of sentences
        count_threshold: minimum number of occurrences for a word to be in the closed vocabulary.

    Returns:
        List of words that appear N times or more

    at this point we will get the dictionary of word count and get rid of some word that happen rar in our document
    and we need some kind of tereshold for identify the limitation of data
    """

    closed_vocab = []

    # Get the word couts of the tokenized sentences
    # Use the function that you defined earlier to count the words
    word_counts = count_words(tokenized_doc)
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # for each word and its count
    for word, cnt in word_counts.items(): # complete this line

        # check that the word's count
        # is at least as great as the minimum count
        if cnt >= count_threshold:

            # append the word to the list
            closed_vocab.append(word)
    ### END CODE HERE ###

    return closed_vocab

In [ ]:
def replace_oov_words_by_unk(tokenized_doc, vocabulary, unknown_token="<unk>"):
    """
    Replace words not in the given vocabulary with '<unk>' token.

    Args:
        tokenized_doc: List of lists of strings
        vocabulary: List of strings that we will use
        unknown_token: A string representing unknown (out-of-vocabulary) words

    Returns:
        List of lists of strings, with words not in the vocabulary replaced

    for having an efficient vocab need to replace some rar word with <unk> token
    """


    vocabulary = set(vocabulary)

    # Initialize a list that will hold the sentences
    # after less frequent words are replaced by the unknown token
    replaced_tokenized_doc = []

    # Go through each sentence
    for doc in tokenized_doc:

        # Initialize the list that will contain
        # a single sentence with "unknown_token" replacements
        replaced_doc = []

        # for each token in the sentence
        for token in doc:

            # Check if the token is in the closed vocabulary
            if token in vocabulary:
                # If so, append the word to the replaced_sentence
                replaced_doc.append(token)
            else:
                # otherwise, append the unknown token instead
                replaced_doc.append(unknown_token)

        # Append the list of tokens to the list of lists
        replaced_tokenized_doc.append(replaced_doc)

    return replaced_tokenized_doc

In [ ]:
def preprocess_data(train_data, test_data, count_threshold):
    """
    Preprocess data, i.e.,
        - Find tokens that appear at least N times in the training data.
        - Replace tokens that appear less than N times by "<unk>" both for training and test data.
    Args:
        train_data, test_data: List of lists of strings.
        count_threshold: Words whose count is less than this are
                      treated as unknown.

    Returns:
        Tuple of
        - training data with low frequent words replaced by "<unk>"
        - test data with low frequent words replaced by "<unk>"
        - vocabulary of words that appear n times or more in the training data
    """

    # Get the closed vocabulary using the train data
    vocabulary = get_words_with_nplus_frequency(train_data,count_threshold)

    # For the train data, replace less common words with "<unk>"
    train_data_replaced = replace_oov_words_by_unk(train_data,vocabulary)

    # For the test data, replace less common words with "<unk>"
    test_data_replaced = replace_oov_words_by_unk(test_data,vocabulary)

    return train_data_replaced, test_data_replaced, vocabulary

In [ ]:
train_data_processed, test_data_processed, vocabulary = preprocess_data(train,
                                                                        test,
                                                                        2)

In [ ]:
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    """
    Count all n-grams in the data

    Args:
        data: List of lists of words
        n: number of words in a sequence

    Returns:
        A dictionary that maps a tuple of n-words to its frequency

    after preprocessing the data we must build our n_gram model
    and we can add <s> and <e> at the start and end of document espetialy when we want work with the sentences
    """

    # Initialize dictionary of n-grams and their counts
    n_grams = {}



    # Go through each sentence in the data
    for sentence in data: # complete this line

        # prepend start token n times, and  append <e> one time
        sentence = [start_token] * n+ sentence + [end_token]
        # convert list to tuple
        # So that the sequence of words can be used as
        # a key in the dictionary
        sentence = tuple(sentence)

        # Use 'i' to indicate the start of the n-gram
        # from index 0
        # to the last index where the end of the n-gram
        # is within the sentence.

        m = len(sentence) if n==1 else len(sentence)-1
        for i in range(m):

            # Get the n-gram from i to i+n
            n_gram = sentence[i:i+n]

            # check if the n-gram is in the dictionary
            if n_gram in n_grams.keys():

                # Increment the count for this n-gram
                n_grams[n_gram] += 1
            else:
                # Initialize this n-gram count to 1
                n_grams[n_gram] = 1

    return n_grams

In [ ]:
uni=count_n_grams(train_data_processed,1)
bi=count_n_grams(train_data_processed,2)
tri=count_n_grams(train_data_processed,3)

In [ ]:
class estimate_probability():

    def __init__(self,n_gram_counts,vocabulary,k=1):

        self.n_gram_counts=n_gram_counts
        self.k=k
        self.vocabulary_size=len(vocabulary)
        self.vocabulary=vocabulary

    def ngram(self,word,previous_n_gram,n_plus1_gram_counts):

        previous_n_gram = tuple(previous_n_gram)


        # Set the denominator
        # If the previous n-gram exists in the dictionary of n-gram counts,
        # Get its count.  Otherwise set the count to zero
        # Use the dictionary that has counts for n-grams
        previous_n_gram_count = self.n_gram_counts[previous_n_gram] if previous_n_gram in self.n_gram_counts  else 0

        # Calculate the denominator using the count of the previous n gram
        # and apply k-smoothing
        denominator = previous_n_gram_count + self.k * self.vocabulary_size

        # Define n plus 1 gram as the previous n-gram plus the current word as a tuple
        n_plus1_gram = previous_n_gram + (word,)

        # Set the count to the count in the dictionary,
        # otherwise 0 if not in the dictionary
        # use the dictionary that has counts for the n-gram plus current word
        n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts  else 0

        # Define the numerator use the count of the n-gram plus current word,
        # and apply smoothing
        numerator = n_plus1_gram_count + self.k

        # Calculate the probability as the numerator divided by denominator
        probability = numerator / denominator

        return probability

    def unigram(self,word):

        numerator=self.n_gram_counts[(word,)] if (word,) in self.n_gram_counts else 0
        numerator+=self.k

        denominator=self.vocabulary_size + self.k * self.vocabulary_size
        probability = numerator / denominator
        return probability


In [ ]:
ob=estimate_probability(uni,vocabulary)

In [ ]:
ob.unigram('در')

0.16820987654320987

In [ ]:
ob.ngaram('در','ورزش',bi)

In [ ]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0,unigram=False):
    """
    Estimate the probabilities of next words using the n-gram counts with k-smoothing

    Args:
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary: List of words
        k: positive constant, smoothing parameter

    Returns:
        A dictionary mapping from next words to the probability.

    here we loop over the entier word
    """
    e_p=estimate_probability(n_gram_counts,vocabulary,k=1)
    # convert list to tuple to use it as a dictionary key
    previous_n_gram = tuple(previous_n_gram)

    # add <e> <unk> to the vocabulary
    # <s> is not needed since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)

    probabilities = {}
    for word in vocabulary:
        if unigram==False:
            probability = e_p.ngram(word,previous_n_gram,n_plus1_gram_counts)
        else:
            probability = e_p.unigram(word)
        probabilities[word] = probability

    return probabilities

In [ ]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):

    """
    now we can custruct the cout matrix for n_gram language model
    Args:
       n_plus1_gram_counts: smooth word


       vocabulary:vocab of words

    """
    # add <e> <unk> to the vocabulary
    # <s> is omitted since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]

    # obtain unique n-grams
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))

    # mapping from n-gram to row
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    # mapping from next word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}

    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count

    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [ ]:
display(make_count_matrix(uni
                          , vocabulary))

,حضور,دومین,خراسان,در,لیگ,برتر,فوتسال,قطعی,شد,.,به,گزارش,دانشجویان,ایران,(,ایسنا,),-,،,امتیاز,تیم,تهران,که,فصل,گذشته,کرد,طور,خریداری,این,با,مالی,پاداش,شده,قرار,است,نام,مسابقات,یابد,حمید,سرمربی,...,حقوقی,تعامل,رعایت,قوانین,المللی,قوانینی,دهند,ان‌ها,می‌گویند,عقب,می‌گردد,بالاخره,زحمت,بگیرند,کنند,معاون,کارها,قاطعانه,لازم,کشور,فلان,وقت,قلم,سیاه,می‌گفت,توانایی,اوردن,پول,می‌خواهد,قایق,دنیامالی,توانمندی,امکانات,رشته‌ها,برخورد,اساسی,طرح,رتبه‌های,<e>,<unk>
(),14.0,3.0,2.0,217.0,9.0,3.0,2.0,2.0,24.0,189.0,148.0,6.0,3.0,18.0,3.0,5.0,3.0,6.0,105.0,2.0,56.0,3.0,128.0,7.0,9.0,25.0,6.0,2.0,114.0,71.0,2.0,2.0,6.0,5.0,59.0,3.0,8.0,2.0,4.0,7.0,...,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,3.0,2.0,2.0,3.0,2.0,3.0,4.0,2.0,2.0,2.0,7.0,3.0,7.0,2.0,3.0,3.0,2.0,3.0,2.0,2.0,4.0,3.0,3.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,10.0,822.0


In [ ]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, vocabulary)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [ ]:
display(make_probability_matrix(bi, vocabulary, k=1))

,حضور,دومین,خراسان,در,لیگ,برتر,فوتسال,قطعی,شد,.,به,گزارش,دانشجویان,ایران,(,ایسنا,),-,،,امتیاز,تیم,تهران,که,فصل,گذشته,کرد,طور,خریداری,این,با,مالی,پاداش,شده,قرار,است,نام,مسابقات,یابد,حمید,سرمربی,...,حقوقی,تعامل,رعایت,قوانین,المللی,قوانینی,دهند,ان‌ها,می‌گویند,عقب,می‌گردد,بالاخره,زحمت,بگیرند,کنند,معاون,کارها,قاطعانه,لازم,کشور,فلان,وقت,قلم,سیاه,می‌گفت,توانایی,اوردن,پول,می‌خواهد,قایق,دنیامالی,توانمندی,امکانات,رشته‌ها,برخورد,اساسی,طرح,رتبه‌های,<e>,<unk>
"(برای,)",0.005839,0.002920,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.004380,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.002920,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.004380,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,...,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.001460,0.004380
"(),)",0.001531,0.001531,0.001531,0.003063,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.003063,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,...,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531,0.001531
"(فنی,)",0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,...,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534
"(باشد,)",0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.003067,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,...,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534,0.001534
"(لازم,)",0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,...,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,0.001522,

In [ ]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0,unigram=False):
    """
    Calculate perplexity for a list of sentences

    Args:
        sentence: List of strings
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary_size: number of unique words in the vocabulary
        k: Positive smoothing constant

    Returns:
        Perplexity score
    """
    # length of previous words
    n = len(list(n_gram_counts.keys())[0])

    # prepend <s> and append <e>
    sentence = ["<s>"] * n + sentence + ["<e>"]

    # Cast the sentence from a list to a tuple
    sentence = tuple(sentence)

    # length of sentence (after adding <s> and <e> tokens)
    N = len(sentence)

    # The variable p will hold the product
    # that is calculated inside the n-root
    # Update this in the code below
    product_pi = 1.0


    # Index t ranges from n to N - 1
    for t in range(n, N):

        # get the n-gram preceding the word at position t
        n_gram = sentence[t-n:t]

        # get the word at position t
        word = sentence[t]

        # Estimate the probability of the word given the n-gram
        # using the n-gram counts, n-plus1-gram counts,
        # vocabulary size, and smoothing constant
        probability = estimate_probability(word,n_gram, n_gram_counts, n_plus1_gram_counts, len(vocabulary_size), k=1,unigram=False)

        # Update the product of the probabilities
        # This 'product_pi' is a cumulative product
        # of the (1/P) factors that are calculated in the loop
        product_pi *= 1 / probability

    # Take the Nth root of the product
    perplexity = product_pi**(1/float(N))


    return perplexity

In [ ]:
#calculate_perplexity for bigram and k smoothing

for doc in test_data_processed:
    print(calculate_perplexity(doc,uni,bi,vocabulary))

In [ ]:
#calculate_perplexity for trigram and k smoothing

for doc in test_data_processed:
    print(calculate_perplexity(doc,bi,tri,vocabulary,1))